# MSDS 631 - Lecture 7 (March 6, 2019)

## Pandas Aggregations and Analytical Methods and Combining Data

### Aggregations

A great deal of analyzing raw data is trying to summarize it for further analysis. So far, we've been writing for-loops and storing data into dictionaries to then run other analyses (think percentage of students on probation). To do this, you defined the attribute you wanted to "group by" (majors, in this case). Pandas allows you to do this automatically and perform certain functions on all of the data associated with each particular value.

If we wanted to use base Python to find the average GPA amongst students in each major, we would do the following:

In [1]:
#Open data
import json
with open('students.json', 'r') as f:
    students_list_of_dicts = json.load(f)

#Create an empty list so we can add the students' GPAs
major_gpas = {}
possible_majors = set([i['major'] for i in students_list_of_dicts])
for major in possible_majors:
    major_gpas[major] = []

#Get all of the students GPAs for their major
for student in students_list_of_dicts:
    student_major = student['major']
    major_gpas[student_major].append(student['gpa'])

#Compute the average
average_gpas = {}
for major in major_gpas:
    avg_gpa = sum(major_gpas[major]) / len(major_gpas[major])
    rounded_gpa = round(avg_gpa, 3)
    average_gpas[major] = rounded_gpa
average_gpas

{'Economics': 3.488,
 'Physics': 3.296,
 'Math': 3.3,
 'Chemistry': 3.359,
 'Engineering': 3.106,
 'Finance': 3.615}

That's **three** separate for-loops with two separate dictionaries that we had to use in order to move data into their appropriate locations so that we could make computations. That's a lot! Imagine what we'd have to do if we wanted to add gender, or worse yet, gender AND class.

With Pandas aggregations we can tell the DataFrame what we want to do with a LOT less code.

Let's start by loading the data into a DataFrame.

In [8]:
import pandas as pd
students_df = pd.read_csv('students.csv')

In [26]:
students_df.head()

,student_id,first,last,gender,class,major,gpa
0,5a397209-3782-4764-a285-10fae807ee71,Janis,Brown,Female,Junior,Economics,3.12
1,e26c3d69-3c74-49b6-81d7-47232787fad9,Timothy,Bishop,Male,Sophomore,Economics,3.48
2,975c1581-5ba2-430c-a3d1-01ce03bd83f9,Elizabeth,Owens,Female,Freshman,Finance,3.40
3,6081f91d-365c-46ce-ad1b-38af120781d9,Edward,Pearson,Male,Freshman,Math,3.84
4,84cec8f4-0b64-44ce-a628-c0eb73f6ca6f,Lisa,Gonzalez,Female,Junior,Finance,4.00


In [27]:
students_df.loc[0]
#loc = location 

student_id    5a397209-3782-4764-a285-10fae807ee71
first                                        Janis
last                                         Brown
gender                                      Female
class                                       Junior
major                                    Economics
gpa                                           3.12
Name: 0, dtype: object

In [28]:
students_df.loc[0, 'student_id']
#want just the value?

'5a397209-3782-4764-a285-10fae807ee71'

In [29]:
#Now let's compute the mean GPA by major
gpa_by_major = students_df.groupby('major')[['gpa']].mean()
gpa_by_major['student_id']

,gpa
major,
Chemistry,3.359151
Economics,3.487805
Engineering,3.106283
Finance,3.614967
Math,3.300452
Physics,3.295663


In [33]:
#Just want the column (a series) #iloc indexes by position loc by name and name can be an integer
gpa_by_major.loc['Economics', 'gpa']

gpa    3.487805
Name: Economics, dtype: float64

In [40]:
gpa_by_major.iloc[1]

gpa    3.487805
Name: Economics, dtype: float64

In [45]:
#Now let's compute the mean GPA by major AND gender dictonary of list gets converted to a dictionary of values gets converted to a series
gpa_by_major_gender = students_df.groupby(['major','gender'])[['gpa']].mean()
gpa_by_major_gender

gpa
major       gender          
Chemistry   Female  3.395772
            Male    3.292617
Economics   Female  3.495707
            Male    3.462748
Engineering Female  2.995098
            Male    3.143491
Finance     Female  3.654529
            Male    3.568535
Math        Female  3.268426
            Male    3.315657
Physics     Female  3.283239
            Male    3.301675

In [48]:
gpa_by_major_gender.loc['Finance'].loc['Female','gpa']
#simpler way below

3.6545289256198346

In [62]:
gpa_by_major_gender.index.tolist() #always access by the index. Column is attribute row is an observation 


[('Chemistry', 'Female'),
 ('Chemistry', 'Male'),
 ('Economics', 'Female'),
 ('Economics', 'Male'),
 ('Engineering', 'Female'),
 ('Engineering', 'Male'),
 ('Finance', 'Female'),
 ('Finance', 'Male'),
 ('Math', 'Female'),
 ('Math', 'Male'),
 ('Physics', 'Female'),
 ('Physics', 'Male')]

In [54]:
gpa_by_major_gender.loc[('Chemistry','Female'),'gpa']

3.395771604938271

In [57]:
#Now let's compute the mean GPY by major, class, and gender
gpa_by_major_class_gender = students_df.groupby(['major','class','gender'])['gpa'].mean()
gpa_by_major_class_gender

major        class      gender
Chemistry    Freshman   Female    3.387490
                        Male      3.286800
             Junior     Female    3.407347
                        Male      3.327231
             Senior     Female    3.401786
                        Male      3.297280
             Sophomore  Female    3.387423
                        Male      3.264516
Economics    Freshman   Female    3.489215
                        Male      3.503711
             Junior     Female    3.478693
                        Male      3.469429
             Senior     Female    3.511972
                        Male      3.477267
             Sophomore  Female    3.499127
                        Male      3.406116
Engineering  Freshman   Female    3.007333
                        Male      3.147173
             Junior     Female    2.973629
                        Male      3.141246
             Senior     Female    3.002258
                        Male      3.155649
             Sophomore 

In [63]:
gpa_by_major_class_gender.loc[['Physics','Senior','Female']]

major    class      gender
Physics  Freshman   Female    3.289189
                    Male      3.315086
         Junior     Female    3.345316
                    Male      3.296198
         Senior     Female    3.234306
                    Male      3.285652
         Sophomore  Female    3.221466
                    Male      3.307823
Name: gpa, dtype: float64

There are many types of computations you can do with aggregations (too many to list here). The most common methods you will call include:
- .mean()
- .max()
- .min()
- .median()
- .size()
 - Counts how many times you see the value of the attribute(s) you are grouping by
- .count()
 - Counts how many non-null values you have in a column
- .rank()
 - Ranks a particular value within a group
 
Let's use the methods above to understand what it's doing

In [70]:
#Max GPA by major
students_df.groupby('major')[['gpa']].max()
                                                

,gpa
major,
Chemistry,4.0
Economics,4.0
Engineering,4.0
Finance,4.0
Math,4.0
Physics,4.0


In [122]:
#Min GPA by major
#col = 'major'
#students_df.groupby(col)[['gpa']].min()

mins = {}
metrics = ['major','class','gender']
for i in metrics:
    mins[i] = students_df.groupby(i)[['gpa']].min()
mins['class']

,gpa
class,
Freshman,2.19
Junior,2.10
Senior,1.72
Sophomore,2.13


In [123]:
#Median GPA by major #a series


In [124]:
#How many students are in each major
students_df.groupby('major').size()

major
Chemistry      1507
Economics      1973
Engineering    2034
Finance        2241
Math            730
Physics        1515
dtype: int64

In [125]:
#How many non-null values are there for each column grouped by major
#Min GPA by major
students_df.groupby('major').count()

,first,last,gender,class,gpa
major,,,,,
Chemistry,1507,1507,1507,1507,1507
Economics,1973,1973,1973,1973,1973
Engineering,2034,2034,2034,2034,2034
Finance,2241,2241,2241,2241,2241
Math,730,730,730,730,730
Physics,1515,1515,1515,1515,1515


In [126]:
#replace entry ?? creating non-null value
students_df.loc(1,'gender')= none:

SyntaxError: invalid syntax (<ipython-input-126-cb12ed30ec66>, line 2)

In [131]:
#changes index to the student id is this error the same
#students_df.set_index('student_id').head()
students_df.set_index('student_id', inplace = True)

KeyError: 'student_id'

In [133]:
students_df.columns

Index(['first', 'last', 'gender', 'class', 'major', 'gpa'], dtype='object')

In [134]:
students_df.head()

,first,last,gender,class,major,gpa
student_id,,,,,,
5a397209-3782-4764-a285-10fae807ee71,Janis,Brown,Female,Junior,Economics,3.12
e26c3d69-3c74-49b6-81d7-47232787fad9,Timothy,Bishop,Male,Sophomore,Economics,3.48
975c1581-5ba2-430c-a3d1-01ce03bd83f9,Elizabeth,Owens,Female,Freshman,Finance,3.40
6081f91d-365c-46ce-ad1b-38af120781d9,Edward,Pearson,Male,Freshman,Math,3.84
84cec8f4-0b64-44ce-a628-c0eb73f6ca6f,Lisa,Gonzalez,Female,Junior,Finance,4.00


In [132]:
#Copute the rank of the students' GPAs, by major: code produces a series.. data frame is a dictionary of lists. 
#Ties are assigned the "best" rank, this is a series, rankins sort form low to high so ascending = false
gpa_ranks = students_df.groupby('major')['gpa'].rank(method='min', ascending=False)
gpa_ranks.head()

student_id
5a397209-3782-4764-a285-10fae807ee71    1675.0
e26c3d69-3c74-49b6-81d7-47232787fad9    1026.0
975c1581-5ba2-430c-a3d1-01ce03bd83f9    1657.0
6081f91d-365c-46ce-ad1b-38af120781d9      32.0
84cec8f4-0b64-44ce-a628-c0eb73f6ca6f       1.0
Name: gpa, dtype: float64

In [135]:
students_df.rename(columns={'gender':'sex'})
students_df.head()

,first,last,gender,class,major,gpa
student_id,,,,,,
5a397209-3782-4764-a285-10fae807ee71,Janis,Brown,Female,Junior,Economics,3.12
e26c3d69-3c74-49b6-81d7-47232787fad9,Timothy,Bishop,Male,Sophomore,Economics,3.48
975c1581-5ba2-430c-a3d1-01ce03bd83f9,Elizabeth,Owens,Female,Freshman,Finance,3.40
6081f91d-365c-46ce-ad1b-38af120781d9,Edward,Pearson,Male,Freshman,Math,3.84
84cec8f4-0b64-44ce-a628-c0eb73f6ca6f,Lisa,Gonzalez,Female,Junior,Finance,4.00


In [129]:
gpa_ranks.head()

student_id
5a397209-3782-4764-a285-10fae807ee71    1675.0
e26c3d69-3c74-49b6-81d7-47232787fad9    1026.0
975c1581-5ba2-430c-a3d1-01ce03bd83f9    1657.0
6081f91d-365c-46ce-ad1b-38af120781d9      32.0
84cec8f4-0b64-44ce-a628-c0eb73f6ca6f       1.0
Name: gpa, dtype: float64

In [ ]:
#fun in place method is x.sort meaning it doesnt, alsmost no method in panda is in place. Grouping and merging changes data so inplace methods are not an option
x = [1,5,2]
x.sort()

#append is also an in place method always returns none

In [130]:
#student_id is indexed
students_df.head()

,first,last,gender,class,major,gpa
student_id,,,,,,
5a397209-3782-4764-a285-10fae807ee71,Janis,Brown,Female,Junior,Economics,3.12
e26c3d69-3c74-49b6-81d7-47232787fad9,Timothy,Bishop,Male,Sophomore,Economics,3.48
975c1581-5ba2-430c-a3d1-01ce03bd83f9,Elizabeth,Owens,Female,Freshman,Finance,3.40
6081f91d-365c-46ce-ad1b-38af120781d9,Edward,Pearson,Male,Freshman,Math,3.84
84cec8f4-0b64-44ce-a628-c0eb73f6ca6f,Lisa,Gonzalez,Female,Junior,Finance,4.00


In [139]:
students_df['gpa_rank'] = gpa_ranks.astype(int) #
students_df.head()

,first,last,gender,class,major,gpa,gpa_rank
student_id,,,,,,,
d3170950-f20b-4121-bd44-2d0595dd46c1,Nancy,Perrodin,Female,Sophomore,Chemistry,4.0,1
13a5fabf-787b-4f40-8609-08366ebae350,Roger,Daise,Male,Freshman,Chemistry,4.0,1
78008297-6fdf-40a5-93eb-f7783d14e41d,Francis,Horn,Male,Sophomore,Chemistry,4.0,1
f82881c7-37ac-432f-b3ee-a032e2ece2cd,Louise,Pendleton,Female,Junior,Chemistry,4.0,1
4ef85dc6-6c78-4961-be4a-44e24b1cd0a6,Joyce,Lopez,Female,Junior,Chemistry,4.0,1


In [142]:
students_df = students_df.sort_values(['major', 'gpa_rank','class'])

In [141]:
students_df.head(20)

,first,last,gender,class,major,gpa,gpa_rank
student_id,,,,,,,
d3170950-f20b-4121-bd44-2d0595dd46c1,Nancy,Perrodin,Female,Sophomore,Chemistry,4.0,1
13a5fabf-787b-4f40-8609-08366ebae350,Roger,Daise,Male,Freshman,Chemistry,4.0,1
78008297-6fdf-40a5-93eb-f7783d14e41d,Francis,Horn,Male,Sophomore,Chemistry,4.0,1
f82881c7-37ac-432f-b3ee-a032e2ece2cd,Louise,Pendleton,Female,Junior,Chemistry,4.0,1
4ef85dc6-6c78-4961-be4a-44e24b1cd0a6,Joyce,Lopez,Female,Junior,Chemistry,4.0,1
baa2901d-a51c-4a3f-b7a5-8d21b8ef1962,Donna,Thompson,Female,Freshman,Chemistry,4.0,1
e336960c-e250-499a-baa7-18ca67657967,Maureen,Hoff,Female,Sophomore,Chemistry,4.0,1
c66bca9c-8f6e-4a20-8463-01f0caba2a35,Maxine,Baker,Female,Freshman,Chemistry,4.0,1
5e195d76-0d73-4633-8b39-7456b491f6bd,Mary,Fernandez,Female,Senior,Chemistry,4.0,1


### Merging Data

Merging data is one of the most powerful tools in Pandas. If you've learned SQL before, then you'll be familiar with a lot of these concepts. Merging allows us to match data from different DataFrames.

Using the students data, imagine we are trying to "standardize" the students GPAs. For those of you unfamiliar with standardization, it is measuring the number of standard deviations away from the mean that a value is.

Since each major has a slightly different level of difficulty and each class has a different composition of student talent, we want to compare each student's GPA against the values for their major and class. Let's do that now.

In [ ]:
mean_gpa_by_major_and_class = 
std_gpa_by_major_and_class = 

In [ ]:
mean_gpa_by_major_and_class.head()

In [ ]:
std_gpa_by_major_and_class.head()

In [ ]:
#Need to rename column since they share the same name - method 1


In [ ]:
#Method 2 for renaming columns
#Note this method RETURNS the new dataframe


In [ ]:
#Merge means to students_df


In [ ]:
#What does the DataFrame look like now?
students_df_w_mean.head()

In [ ]:
#Merge standard deviations to new students_df


In [ ]:
#What does the DataFrame look like now?
students_df_w_mean_std.head()

In [ ]:
#Compute how far from the mean the student's GPA is
students_df_w_mean_std['std_from_mean'] = 

In [ ]:
#Compute how many standard deviations away from the mean the student's GPA is
students_df_w_mean_std['standardized_gpa'] = 

In [ ]:
#Look at the new data
students_df_w_mean_std.head()

In [ ]:
#Look at the distribution of original GPAs
from matplotlib import pyplot as plt
students_df_w_mean_std['gpa'].hist(bins=20)
plt.show()

In [ ]:
#Look at the distribution of standardized GPAs
students_df_w_mean_std['standardized_gpa'].hist(bins=20)
plt.show()

Let's try looking at disparate data and use it to join the data.

Let's use Pandas to get all of the data from Quiz 2 into the same DataFrame.

In [ ]:
with open('department_enrollment.json', 'r') as f:
    dept_enrollment = json.load(f)

In [ ]:
dept_enrollment.keys()

In [ ]:
#Let's try creating a DataFrame from this dictionary of lists


The creation of DataFrames from dictionaries of lists MUST have lists that are the same length. Since we can't do that with our dictionary here, we're going to have to manually do this.

In [ ]:
#Write a function that makes a single DataFrame for a major


In [ ]:
#Let's create a list of DataFrames and concatenate them together


In [ ]:
#Take a look at the new big DataFrame


In [ ]:
#Let's load the student_gpas.json file


In [ ]:
#Getting the data into a DataFrame isn't easy


In [ ]:
#Load student_directory data


In [ ]:
#Now let's combine all of the data together
